In [1]:
# import all libraries
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Conv1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import spacy
nlp=spacy.load("en_core_web_sm")

In [2]:
# Download dataset
# https://github.com/crwong/cs224u-project/blob/master/data/sentiment/training.1600000.processed.noemoticon.csv

In [3]:
#load the dataset
train=pd.read_csv("training.1600000.processed.noemoticon.csv" , encoding= "latin-1")
Y_train = train[train.columns[0]]
X_train = train[train.columns[5]]

In [4]:
# split the data into test and train
from sklearn.model_selection import train_test_split
trainset1x, trainset2x, trainset1y, trainset2y = train_test_split(X_train.values, Y_train.values, test_size=0.02,random_state=42 )
trainset2y=pd.get_dummies(trainset2y)

In [5]:
len(trainset1x), len(trainset2x)

(19599, 400)

In [6]:
# function to remove stopwords
def stopwords(sentence):
    new=[]
    sentence=nlp(sentence)
    for w in sentence:
        if (w.is_stop == False) & (w.pos_ !="PUNCT"):
            new.append(w.string.strip())
        c=" ".join(str(x) for x in new)
    return c

In [7]:
# function to lemmatize the tweets
def lemmatize(sentence):
    sentence=nlp(sentence)
    str=""
    for w in sentence:
        str+=" "+w.lemma_
    return nlp(str)

In [8]:
#loading the glove model
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r', encoding="utf8")
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = [float(val) for val in splitLine[1:]]
        model[word] = embedding
    print ("Done."),len(model),(" words loaded!")
    return model

In [9]:
# save the glove model
model=loadGloveModel("glove.twitter.27B.200d.txt")

Loading Glove Model
Done.


In [10]:
trainset2x[1]

'The first serial about my city so stupid... '

In [11]:
#vectorising the sentences
def sent_vectorizer(sent, model):
    sent_vec = np.zeros(200)
    numw = 0
    for w in sent.split():
        try:
            sent_vec = np.add(sent_vec, model[str(w)])
            numw+=1
        except:
            pass
    return sent_vec

In [12]:
#obtain a clean vector
cleanvector=[]
for i in range(trainset2x.shape[0]):
    document=trainset2x[i]
    document=document.lower()
    document=lemmatize(document)
    document=str(document)
    cleanvector.append(sent_vectorizer(document,model))

In [13]:
len(cleanvector), len(cleanvector[0])

(400, 200)

In [14]:
#Getting the input and output in proper shape
cleanvector=np.array(cleanvector)
print(cleanvector.shape)
len(cleanvector[0])

(400, 200)


200

In [15]:
#Getting the input and output in proper shape
cleanvector =cleanvector.reshape(len(cleanvector),200,1)
print(cleanvector.shape)
len(cleanvector[0])

(400, 200, 1)


200

In [16]:
trainset2x[0:2]

array(['2 more Exams ', 'The first serial about my city so stupid... '],
      dtype=object)

In [17]:
#tokenizing the sequences
tokenizer = Tokenizer(num_words=16000)
tokenizer.fit_on_texts(trainset2x)
sequences = tokenizer.texts_to_sequences(trainset2x)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=15, padding="post")
print(data.shape, len(data))

Found 1873 unique tokens.
(400, 15) 400


In [18]:
#reshape the data and preparing to train
data=data.reshape(len(data),15,1)
from sklearn.model_selection import train_test_split
trainx, validx, trainy, validy = train_test_split(data, trainset2y, test_size=0.3,random_state=42 )

In [19]:
trainx.shape

(280, 15, 1)

In [20]:
#calculate the number of words
nb_words=len(tokenizer.word_index)+1
nb_words

1874

In [21]:
#obtain theembedding matrix
embedding_matrix = np.zeros((nb_words, 200))
for word, i in word_index.items():
    embedding_vector = model.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Null word embeddings: 346


In [22]:
embedding_matrix.shape

(1874, 200)

In [23]:
trainy=np.array(trainy)
validy=np.array(validy)

In [24]:
#building a simple RNN model
def modelbuild():
    model = Sequential()
    model.add(keras.layers.InputLayer(input_shape=(15,1)))
    keras.layers.Embedding(input_dim=nb_words, output_dim=15, weights=[embedding_matrix], input_length=15,trainable=False)
    model.add(keras.layers.SimpleRNN(units = 100, activation='relu',use_bias=True))
    model.add(keras.layers.Dense(units=1000, input_dim = 2000, activation='sigmoid'))
    model.add(keras.layers.Dense(units=500, input_dim=1000, activation='relu'))
    model.add(keras.layers.Dense(units=2, input_dim=500,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [25]:
#compiling the model
finalmodel = modelbuild()
finalmodel.fit(trainx, trainy, epochs=10, batch_size=120,validation_data=(validx,validy))

Epoch 1/10
3/3 [==============================] - 2s 234ms/step - loss: 2.8442 - accuracy: 0.4750 - val_loss: 1.1160 - val_accuracy: 0.5083
Epoch 2/10
3/3 [==============================] - 0s 43ms/step - loss: 1.3007 - accuracy: 0.5321 - val_loss: 0.8702 - val_accuracy: 0.5167
Epoch 3/10
3/3 [==============================] - 0s 36ms/step - loss: 0.7232 - accuracy: 0.5429 - val_loss: 0.9956 - val_accuracy: 0.4917
Epoch 4/10
3/3 [==============================] - 0s 41ms/step - loss: 0.8889 - accuracy: 0.4679 - val_loss: 0.7325 - val_accuracy: 0.4750
Epoch 5/10
3/3 [==============================] - 0s 32ms/step - loss: 0.7064 - accuracy: 0.5536 - val_loss: 0.8442 - val_accuracy: 0.5167
Epoch 6/10
3/3 [==============================] - 0s 43ms/step - loss: 0.7437 - accuracy: 0.5821 - val_loss: 0.7323 - val_accuracy: 0.5167
Epoch 7/10
3/3 [==============================] - 0s 50ms/step - loss: 0.6425 - accuracy: 0.6464 - val_loss: 0.7377 - val_accuracy: 0.5083
Epoch 8/10
3/3 [==========